In [4]:
%%capture

!pip install cirq
!pip install xlrd
!pip install pysf
!pip install openfermion
!pip install openfermionpyscf
import numpy as np

In [5]:
import cirq
import scipy
import pyscf
import openfermion
import openfermionpyscf 
import itertools

from pyscf import gto, scf, fci
from openfermion.transforms import jordan_wigner, get_fermion_operator
from openfermionpyscf import run_pyscf
from openfermion.utils import count_qubits
from openfermion.linalg import jw_hartree_fock_state
from openfermion.circuits import simulate_trotter
from openfermion import (
    get_sparse_operator, get_ground_state, FermionOperator,
    jw_get_ground_state_at_particle_number, MolecularData,
    expectation, uccsd_convert_amplitude_format,
    get_interaction_operator, QubitOperator, eigenspectrum,
    InteractionOperator
)

## Function Definitions

In [6]:
chemicalAccuracy = 1.5936*10**-3

# Define necessary Pauli operators (two-dimensional) as matrices
pauliX = np.array([[0,1],
                 [1,0]],
                dtype = complex)
pauliZ = np.array([[1,0],
                 [0,-1]],
                dtype = complex)
pauliY = np.array([[0,-1j],
                 [1j,0]],
                dtype = complex)

def stringToMatrix(pauliString):
  '''
  Converts a Pauli string to its matrix form.

  Arguments:
    pauliString (str): the Pauli string (e.g. "IXYIZ")

  Returns:
    matrix (np.ndarray): the corresponding matrix, in the computational basis

  '''

  matrix = np.array([1])

  # Iteratively construct the matrix, going through each single qubit Pauli term
  for pauli in pauliString:
      if pauli == "I":
        matrix = np.kron(matrix,np.identity(2))
      elif pauli == "X":
        matrix = np.kron(matrix,pauliX)
      elif pauli == "Y":
        matrix = np.kron(matrix,pauliY)
      elif pauli == "Z":
        matrix = np.kron(matrix,pauliZ)

  return matrix
  
def fromVectortoKet(stateVector):
  '''
  Transforms a vector representing a basis state to the corresponding ket.

  Arguments:
    stateVector (np.ndarray): basis vector in the 2^n-dimensional Hilbert space

  Returns:
    ket (list): a list of length n representing the corresponding ket 
  '''

  dim = len(stateVector)
  ket = []

  while dim>1:
    if any (stateVector[i] for i in range(int(dim/2))):
      # Ket is of the form |0>|...>. 

      #Fix |0> as the msq.
      ket.append(0)

      # Get the vector representing the state of the remaining qubits.
      stateVector = stateVector[:int(dim/2)]

    else:
      # Ket is of the form |1>|...>. 
      
      #Fix |0> as the msq.
      ket.append(1)

      # Get the vector representing the state of the remaining qubits.
      stateVector = stateVector[int(dim//2):]

    dim = dim/2

  return ket

def fromKettoVector(ket):
  '''
  Transforms a ket representing a basis state to the corresponding state vector.

  Arguments:
    ket (list): a list of length n representing the ket 

  Returns:
    stateVector (np.ndarray): the corresponding basis vector in the 
      2^n dimensional Hilbert space
  '''
  stateVector = [1]

  # Iterate through the ket, calculating the tensor product of the qubit states
  for i in ket:
    qubitVector = [not i,i]
    stateVector = np.kron(stateVector,qubitVector)

  return stateVector

def calculateOverlap(stateCoordinates1,stateCoordinates2):
    '''
    Calculates the overlap between two states, given their coordinates.

    Arguments:
      stateCoordinates1 (np.ndarray): the coordinates of one of the states in 
        some orthonormal basis,
      stateCoordinates2 (np.ndarray): the coordinates of the other state, in 
        the same basis

    Returns: 
      overlap (float): the overlap between two states (absolute value of the 
        inner product).
    '''

    bra = np.conj(stateCoordinates1)
    ket = stateCoordinates2
    overlap = np.abs(np.dot(bra,ket))
    
    return overlap

def findSubStrings(mainString,hamiltonian,checked = []):
    '''
    Finds and groups all the strings in a Hamiltonian that only differ from 
    mainString by identity operators.

    Arguments:
      mainString (str): a Pauli string (e.g. "XZ)
      hamiltonian (dict): a Hamiltonian (with Pauli strings as keys and their 
        coefficients as values)
      checked (list): a list of the strings in the Hamiltonian that have already
        been inserted in another group

    Returns: 
      groupedOperators (dict): a dictionary whose keys are boolean strings 
        representing substrings of the mainString (e.g. if mainString = "XZ", 
        "IZ" would be represented as "01"). It includes all the strings in the 
        hamiltonian that can be written in this form (because they only differ 
        from mainString by identities), except for those that were in checked
        (because they are already part of another group of strings).
      checked (list):  the same list passed as an argument, with extra values
        (the strings that were grouped in this function call).
    '''
    
    groupedOperators = {}
    
    # Go through the keys in the dictionary representing the Hamiltonian that 
    #haven't been grouped yet, and find those that only differ from mainString 
    #by identities
    for pauliString in hamiltonian:
        
        if pauliString not in checked:
            # The string hasn't been grouped yet
            
            if(all((op1 == op2 or op2 == "I") \
                   for op1,op2 in zip(mainString,pauliString))):
                # The string only differs from mainString by identities
                
                # Represent the string as a substring of the main one
                booleanString = "".join([str(int(op1 == op2)) for op1,op2 in \
                                       zip(mainString,pauliString)])
                    
                # Add the boolean string representing this string as a key to 
                #the dictionary of grouped operators, and associate its 
                #coefficient as its value
                groupedOperators[booleanString] = hamiltonian[pauliString]
                
                # Mark the string as grouped, so that it's not added to any 
                #other group
                checked.append(pauliString)
                
    return (groupedOperators,checked)

def groupHamiltonian(hamiltonian):
    '''
    Organizes a Hamiltonian into groups where strings only differ from 
    identities, so that the expectation values of all the strings in each 
    group can be calculated from the same measurement array.

    Arguments: 
      hamiltonian (dict): a dictionary representing a Hamiltonian, with Pauli 
        strings as keys and their coefficients as values.

    Returns: 
      groupedHamiltonian (dict): a dictionary of subhamiltonians, each of 
        which includes Pauli strings that only differ from each other by 
        identities. 
        The keys of groupedHamiltonian are the main strings of each group: the 
        ones with least identity terms. The value associated to a main string is 
        a dictionary, whose keys are boolean strings representing substrings of 
        the respective main string (with 1 where the Pauli is the same, and 0
        where it's identity instead). The values are their coefficients.
    '''
    groupedHamiltonian = {}
    checked = []
    
    # Go through the hamiltonian, starting by the terms that have less
    #identity operators
    for mainString in \
        sorted(hamiltonian,key = lambda pauliString: pauliString.count("I")):
            
        # Call findSubStrings to find all the strings in the dictionary that 
        #only differ from mainString by identities, and organize them as a 
        #dictionary (groupedOperators)
        groupedOperators,checked = findSubStrings(mainString,hamiltonian,checked)
        
        # Use the dictionary as a value for the mainString key in the 
        #groupedHamiltonian dictionary
        groupedHamiltonian[mainString] = groupedOperators
        
        # If all the strings have been grouped, exit the for cycle
        if(len(checked) == len(hamiltonian.keys())):
           break
       
    return groupedHamiltonian 
    
def convertHamiltonian(openfermionHamiltonian):
  '''
  Formats a qubit Hamiltonian obtained from openfermion, so that it's a suitable
  argument for functions such as measureExpectationEstimation.

  Arguments:
    openfermionHamiltonian (openfermion.qubitOperator): the Hamiltonian.

  Returns:
    formattedHamiltonian (dict): the Hamiltonian as a dictionary with Pauli
      strings (eg 'YXZI') as keys and their coefficients as values.
  '''

  formattedHamiltonian = {}
  qubitNumber = count_qubits(openfermionHamiltonian)

  # Iterate through the terms in the Hamiltonian
  for term in openfermionHamiltonian.get_operators():

    operators = []
    coefficient = list(term.terms.values())[0]
    pauliString = list(term.terms.keys())[0]
    previousQubit = -1

    for (qubit,operator) in pauliString:

      # If there are qubits in which no operations are performed, add identities 
      #as necessary, to make sure that the length of the string will match the 
      #number of qubits
      identities = (qubit-previousQubit-1)
      if identities>0: 
        operators.append('I'*identities)

      operators.append(operator)
      previousQubit = qubit
    
    # Add final identity operators if the string still doesn't have the 
    #correct length (because no operations are performed in the last qubits)
    operators.append('I'*(qubitNumber-previousQubit-1))

    formattedHamiltonian["".join(operators)] = coefficient

  return formattedHamiltonian

def hamiltonianToMatrix(hamiltonian):
    '''
    Convert a Hamiltonian (from OpenFermion) to matrix form.
    
    Arguments:
      hamiltonian (openfermion.InteractionOperator): the Hamiltonian to be
        transformed.

    Returns:
      matrix (np.ndarray): the Hamiltonian, as a matrix in the computational 
        basis
    
    ''' 
    
    qubitNumber = hamiltonian.n_qubits
    
    hamiltonian = jordan_wigner(hamiltonian)
    formattedHamiltonian = convertHamiltonian(hamiltonian)
    groupedHamiltonian = groupHamiltonian(formattedHamiltonian)

    matrix = np.zeros((2**qubitNumber,2**qubitNumber),dtype = complex)

    # Iterate through the strings in the Hamiltonian, adding the respective 
    #contribution to the matrix
    for string in groupedHamiltonian:

      for substring in groupedHamiltonian[string]:
        pauli = ("".join("I"*(not int(b)) + a*int(b) \
                         for (a,b) in zip(string,substring)))
        
        matrix += stringToMatrix(pauli) * groupedHamiltonian[string][substring]

    return matrix

def groundStatesFromDiagonalization(listOfHamiltonians):
    '''
    Obtains the ground states and energies corresponding to a list of 
    Hamiltonians, by performing exact diagonalization.

    Arguments:
      listOfHamiltonians (list): the list of hamiltonians as dictionaries.

    Returns: 
      exactGroundEnergies (list): the list of ground energies corresponding to
        the list of Hamiltonians
      exactGroundStates (list): the list of the ground states corresponding to 
        the list of hamiltonians
    diagonalization.
    '''
    
    exactGroundEnergies = []
    exactGroundStates = []
    
    # Obtain the number of qubits and calculate the dimension.
    n = len(list(listOfHamiltonians[0].keys())[0])
    dim = n**2

    # Find the ground state and energy of each Hamiltonian in the list.
    for hamiltonian in listOfHamiltonians:
        
        # Get the matrix form of the Hamiltonian, in the computational basis.
        hamiltonianMatrix = np.zeros((dim,dim),dtype = complex)

        for pauliString in hamiltonian:
            coefficient = hamiltonian[pauliString]
            hamiltonianMatrix += coefficient * stringToMatrix(pauliString)
            
        w,v = np.linalg.eigh(hamiltonianMatrix)
        
        # Add the minimum eigenvalue of the Hamiltonian to the list of ground
        #energies
        exactGroundEnergies.append((np.amin(w)))

        # Add the corresponging eigenstate to the list of ground states
        exactGroundStates.append(v[np.argmin(w)])
        
    return (exactGroundEnergies,exactGroundStates)

def unitaryToRotationGates(unitaryMatrix,qubit,keepPhase = False):
    ''' 
    Decomposes a single qubit unitary matrix into rotation gates.

    Arguments: 
      unitaryMatrix (np.ndarray): the matrix to be decomposed
      qubit (cirq.LineQubit): the qubit the matrix acts on
      keepPhase (bool): a flag indicating whether the global phase should be 
        kept

    Returns: 
      (list): a list of single qubit rotation gates, equivalent to the action of 
        unitaryMatrix (up to a global phase, if keepPhase = False)
    ''' 
    
    # Compute the angles of each rotation from the entries of the matrix
    rz1angle = -np.angle(unitaryMatrix[0][0])+np.angle(-unitaryMatrix[0][1])
    ryangle = 2*np.arccos(np.abs(unitaryMatrix[0][0]))
    rz2angle = -np.angle(unitaryMatrix[0][0])+np.angle(unitaryMatrix[1][0])
    
    if keepPhase:
        gp = np.angle(unitaryMatrix[0][0])+(rz1angle+rz2angle)/2
    
    # Create the necessary rotation gates
    r1 = cirq.rz(rz1angle)
    r2 = cirq.ry(ryangle)
    r3 = cirq.rz(rz2angle)
    
    yield [r1(qubit),r2(qubit),r3(qubit)]

    if keepPhase:
        # The global phase should be kept; add the correction.
        yield cirq.MatrixGate(np.identity(2)*np.exp(1j*gp)).on(qubit)

def measureString(pauliString,repetitions,statePreparationGates,qubits):
    ''' 
    Measures the expectation value of a Pauli string using the CIRQ simulator 
    (simulating sampling).

    Arguments:
      pauliString (str): the Pauli string to be measured.
      repetitions (int): the number of circuit repetitions to be used.
      statePreparationGates (list): the list of CIRQ gates that prepare the 
        state in which to obtain the expectation value.

    Returns: 
      expecationValue (float): the expectation value of pauliString, with 
        sampling noise.
    ''' 

    # Initialize circuit.
    circuit = cirq.Circuit(statePreparationGates)

    # Optimize circuit.
    cirq.optimizers.EjectZ().optimize_circuit(circuit)
    cirq.optimizers.DropNegligible().optimize_circuit(circuit)
    
    # Append necessary rotations and measurements for each qubit.
    for i in range(len(qubits)):
        op = pauliString[i]
        
        # Rotate qubit i to the X basis if that's the desired measurement.
        if (op == "X"):
            circuit.append(cirq.H(qubits[i]))
        
        # Rotate qubit i to the Y basis if that's the desired measurement.
        if (op == "Y"):
            # Apply adjoint of the Clifford S gate
            circuit.append(cirq.ops.ZPowGate(exponent = -1/2).on(qubits[i]))

            # Finish rotating to Y basis, proceeding as for X
            circuit.append(cirq.H(qubits[i]))
            
        # Measure qubit i in the computational basis, unless operator is I.
        if (op != "I"):
            circuit.append(cirq.measure(qubits[i],key = str(i)))
            
    # Sample the desired number of repetitions from the circuit, unless
    #there are no measurements (identity term).
    if (pauliString != "I"*len(qubits)):
        s = cirq.Simulator()
        results = s.run(circuit,repetitions = repetitions)
    #print(circuit)

    # Calculate the expectation value of the Pauli string by averaging over  
    #all the repetitions.
    
    total = 0
    
    for j in range(repetitions):
        meas = 1
        for i in range(len(qubits)):
            if (pauliString[i] != "I"):
                meas = meas*(1-2*results.data[str(i)][j])
        total += meas
        
    expectationValue = total/repetitions
    
    return expectationValue

def measureExpectation(mainString,subHamiltonian,repetitions,\
                       statePreparationGates,qubits):
    ''' 
    Measures the expectation value of a subHamiltonian using the CIRQ simulator 
    (simulating sampling). By construction, all the expectation values of the 
    strings in subHamiltonian can be obtained from the same measurement array.

    Arguments: 
      mainString (str): the main Pauli string. This is the string in the group
        with the least identity terms. It defines the circuit that will be used.
      subHamiltonian (dict): a dictionary whose keys are boolean strings
        representing substrings of the main one, and whose values are the 
        respective coefficients.
      repetitions (int): the number of repetitions to be performed, the 
      statePreparationGates (list): the list of CIRQ gates that prepare (from 
        |0..0>) the state in which to obtain the expectation value.
      qubits (list): list of cirq.LineQubit to apply the gates on

    Returns:
      totalExpectationValue (float): the total expectation value of 
        subHamiltonian, with sampling noise.
    ''' 
    
    # Initialize circuit.
    circuit = cirq.Circuit()
    
    # Append to the circuit the gates that prepare the state corresponding to
    #the received parameters.
    circuit.append(statePreparationGates)
    cirq.optimizers.EjectZ().optimize_circuit(circuit)
    cirq.optimizers.DropNegligible().optimize_circuit(circuit)
    
    # Append necessary rotations and measurements for each qubit.
    for i in range(len(qubits)):
        op = mainString[i]
        
        # Rotate qubit i to the X basis if that's the desired measurement.
        if (op == "X"):
            circuit.append(cirq.H(qubits[i]))
            
        # Rotate qubit i to the X basis if that's the desired measurement.
        if (op == "Y"):
            # Apply adjoint of the Clifford S gate
            circuit.append(cirq.ops.ZPowGate(exponent = -1/2).on(qubits[i]))

            # Apply the Hadamard gate
            circuit.append(cirq.H(qubits[i]))
            
        #Measure qubit i in the computational basis, unless operator is I.
        if (op != "I"):
            circuit.append(cirq.measure(qubits[i],key = str(i)))
            
    # Sample the desired number of repetitions from the circuit, unless
    #there are no measurements (identity term).
    if (mainString != "I"*len(qubits)):
        s = cirq.Simulator()
        results = s.run(circuit,repetitions = repetitions)

    # For each substring, initialize the sum of all measurements as zero
    total = {}
    for subString in subHamiltonian:
        total[subString] = 0
    
    # Calculate the expectation value of each Pauli string by averaging over  
    #all the repetitions
    for j in range(repetitions):
        meas = {}
        
        # Initialize the measurement in repetition j for all substrings
        for subString in subHamiltonian:
            meas[subString] = 1
        
        # Go through the measurements on all the qubits
        for i in range(len(qubits)):
            
            if (mainString[i] != "I"):
                # There's a measurement associated with this qubit
                
                # Use this single qubit measurement for the calculation of the
                #measurement of each full substring in this repetition. If the
                #substring has a "0" in the position corresponding to this
                #qubit, the operator associated is I, and the measurement
                #is ignored (raised to the power of 0)
                for subString in subHamiltonian:
                    meas[subString] = meas[subString]*((1-2*results.data[str(i)][j])\
                                                     **int(subString[i]))
                        
        # Add this measurement to the total, for each string
        for subString in subHamiltonian:
            total[subString]+=meas[subString]
        
    totalExpectationValue = 0
    
    # Calculate the expectation value of the subHamiltonian, by multiplying
    #the expectation value of each substring by the respective coefficient
    for subString in subHamiltonian:
        
        # Get the expectation value of this substring by taking the average
        #over all the repetitions
        expectationValue = total[subString]/repetitions
        
        # Add this value to the total expectation value, weighed by its 
        #coefficient
        totalExpectationValue+=expectationValue*subHamiltonian[subString]
    
    return(totalExpectationValue)

count = 0
trackOptimization = False

def stateEnergy(statePreparationGates,hamiltonian,repetitions,qubits):
    ''' 
    Returns: the experimental energy expectation in a given state.
    Arguments: a list of 6 parameters defining the state in which to obtain the 
    expectation value, a dictionary with the coefficients of each Pauli string
    term in the Hamiltonian, the number of repetitions to be used to calculate 
    the expectation value of each string.
    ''' 
    
    # Print the percentage of the maximum number of function evaluations that 
    #has been used so far in the classical optimization, if it's a multiple of
    #10% (to inform on the stage of the optimization process).
    global count, trackOptimization
    if(trackOptimization):
        if (count%(300/10) == 0):
            print(round(count/(300/100)),"%",sep = '')
        count = count+1

    groupedHamiltonian = groupHamiltonian(hamiltonian)

    experimentalEnergyExpectation = 0
    
    # Obtain experimental expectation value for each necessary Pauli string by
    #calling the measureExpectation function, and perform the necessary weighed
    #sum to obtain the energy expectation value.
    for mainString in groupedHamiltonian:
         
        expectationValue = measureExpectation(mainString,\
                                            groupedHamiltonian[mainString],\
                                                repetitions,statePreparationGates,qubits)
        experimentalEnergyExpectation+=expectationValue

    return experimentalEnergyExpectation

def exactStateEnergy(stateCoordinates,hamiltonian):
    ''' 
    Calculates the exact energy in a specific state.

    Arguments:
      stateCoordinates (np.ndarray): the state in which to obtain the 
        expectation value.
      hamiltonian (dict): the Hamiltonian of the system.
    
    Returns:
      exactEnergy (float): the energy expecation value in the state.
    ''' 

    exactEnergy = 0
    
    # Obtain the theoretical expectation value for each Pauli string in the
    #Hamiltonian by matrix multiplication, and perform the necessary weighed
    #sum to obtain the energy expectation value.
    for pauliString in hamiltonian:
        
        ket = np.array(stateCoordinates,dtype = complex)
        bra = np.conj(ket)
        
        ket = np.matmul(stringToMatrix(pauliString),ket)
        expectationValue = np.real(np.dot(bra,ket))
        
        exactEnergy+=\
            hamiltonian[pauliString]*expectationValue
            
    return exactEnergy
    
def exactStateEnergySparse(stateVector,sparseHamiltonian):
    ''' 
    Calculates the exact energy in a specific state, using sparse matrices.

    Arguments:
      stateVector (Union[np.ndarray, scipy.sparse.csc_matrix): the state in 
        which to obtain the expectation value.
      sparseHamiltonian (scipy.sparse.csc_matrix): the Hamiltonian of the system.
    
    Returns:
      energy (float): the energy expecation value in the state.
    ''' 

    if not isinstance(stateVector,scipy.sparse.csc_matrix):
      ket = scipy.sparse.csc_matrix(stateVector,dtype=complex).transpose()
    else:
      ket = stateVector
      
    bra = ket.transpose().conj()

    energy = (bra * sparseHamiltonian * ket)[0,0].real
    
    return energy
    
def energyFromExpOperator(operator,referenceState,sparseHamiltonian):
  '''
  Calculates the energy in the state obtained by applying e^operator to the
  reference state, using sparse matrices.

  Arguments:
    operator (openfermion.FermionOperator): the operator to be exponentiated
      and applied to the reference state.
    referenceState (np.ndarray): the reference state.
    sparseHamiltonian (scipy.sparse.csc_matrix): the Hamiltonian of the system,
      in matrix form.

  Returns:
    energy (float): the energy.
  '''
  # Obtain the dimension and, from it, calculate the number of qubits
  dimension, _ = sparseHamiltonian.shape
  qubitNumber = int(np.log(dimension)/np.log(2)) 
  
  # Obtain the sparse matrix representing the operator, and act on the state
  #with its exponentiated version
  sparseOperator = get_sparse_operator(operator,qubitNumber)
  ket = scipy.sparse.csc_matrix(referenceState,dtype=complex).transpose()
  ket = scipy.sparse.linalg.expm_multiply(sparseOperator,ket)

  # Calculate the energy expectation value
  bra = ket.transpose().conj()
  energy = (bra * sparseHamiltonian * ket)[0,0].real

  return energy

## Molecule Definitions 

In [7]:
# H2
geometry = [['H',[0,0,0]],['H',[0,0,0.74]]]
basis = 'sto-3g'
multiplicity = 1
charge = 0
h2molecule = MolecularData(geometry,basis,multiplicity,charge)
h2molecule = run_pyscf(h2molecule,run_fci = True,run_ccsd = True)

# HeH+
r = 1 # interatomic distance in angstrom
geometry = [['He',[0,0,0]],['H',[0,0,r]]]
basis = 'sto-3g'
multiplicity = 1
charge = +1
helonium = MolecularData(geometry,basis,multiplicity,charge)
helonium = run_pyscf(helonium,run_fci = True,run_ccsd = True)

# Get the PySCF molecule as well (it's not necessary as there is the OF-PySCF
#plugin, just for confirmation)
helonium_pysf = gto.M(atom = geometry, charge = charge, basis = basis,verbose = 0)

# LiH
bondLength = 1.45 # interatomic distance in angstrom
geometry = [['Li',[0,0,0]],['H',[0,0,bondLength]]]
basis = 'sto-3g'
multiplicity = 1
charge = 0
liH = MolecularData(geometry,basis,multiplicity,charge)
liH = run_pyscf(liH,run_fci = True,run_ccsd = True)

# Alternative to using run_pyscf: load from OpenFermion (data for this 
#particular molecule at this particular interatomic distance is available in 
#a file that comes with OF)
liHOF = MolecularData(geometry,basis,multiplicity,charge,description = '1.45')
liHOF.load()

## Choosing the molecule

In [8]:
molecule = h2molecule

In [9]:
qubitNumber = molecule.n_qubits
electronNumber = molecule.n_electrons
orbitalNumber = molecule.n_orbitals

hfState = jw_hartree_fock_state(electronNumber,orbitalNumber*2) 

hamiltonian = molecule.get_molecular_hamiltonian() 
sparseHamiltonian = get_sparse_operator(hamiltonian)
qubitHamiltonian = jordan_wigner(get_fermion_operator(hamiltonian))
formattedHamiltonian = convertHamiltonian(qubitHamiltonian)

## Estimating the required shot count

See [arxiv:1507.08969](https://https://arxiv.org/pdf/1507.08969.pdf). We write the Hamiltonian as a sum of Pauli strings, as usual:

$H = \sum_{i=1}^{N_{terms}}h_iO_i$

The error coming from the finite number of shots in the expectation estimation can be written

$\epsilon^2 = \sum_i\frac{|h_i|^2Var(O_i)}{M_i}$



The variances depend, not only on the Pauli operators appearing in the Hamiltonian, but also on the particular state. We can bound them as $Var(O_i)\leq1$, since any Pauli operator has $\pm1$ eigenvalues and so will their product.

Without further knowledge about their variance, the best choice is to distribute the shots by the operators proportionatelly to the norm of their coefficients: $M_i\propto|h_i|$. Assuming this optimal distribution, one can estimate the necessary number of shots to hit a precision $\epsilon$ as

$M\approx\frac{(\sum_i|h_i|)^2}{\epsilon^2}$

The term $(\sum_i|h_i|)^2$ will depend strongly on the molecule in study, and make it so that bigger molecules require a larger number of shots for the same precision.

Given that chemical accuracy is $1$ kcal/mol $\approx 1.59\times10^{-3}$ Hartree, achieving it requires

$M\approx0.4\times10^6\times(\sum_i|h_i|)^2$

This is already of the order of $10^7$ to $10^8$ for small molecules like helonium and lithium hydride; $Fe_2S_2$, for instance, would require $10^{13}$ shots.

This number of shots needs to be performed every time one needs to evaluate the energy, which happens many times throughout the optimization - especially for larger molecules, in which there are many parameters to optimize. And it still doesn't guarantee proper functionning of the optimizer: it guarantees chemical accuracy in the energy estimates, but the final estimate isn't necessarily close to the ground state. The error might still be big enough to confuse the optimizer into picking a wrong guess.

In [10]:
total = 0

for term in qubitHamiltonian.get_operators():
  coefficient = list(term.terms.values())[0]
  total += np.abs(coefficient)

shots = total**2 / (1.59 * 10**(-3)) ** 2
print("Shots required to hit chemical accuracy:",int(shots))

Shots required to hit chemical accuracy: 1557274


## The UCCSD Operator

### Getting the operator from OpenFermion

In [11]:
ccsdSingleAmps = molecule.ccsd_single_amps
ccsdDoubleAmps = molecule.ccsd_double_amps
packedAmplitudes = openfermion.uccsd_singlet_get_packed_amplitudes\
  (ccsdSingleAmps,ccsdDoubleAmps,qubitNumber,electronNumber)

# Get the final state and energy with Hartree Fock circuit + UCCSD circuit
#with CCSD amplitudes. These exact values are useful to compare against 
#those resulting from trotterized versions of the operators.

uccsdOperator = openfermion.uccsd_singlet_generator\
  (packedAmplitudes,qubitNumber,electronNumber)

sparseOperator = get_sparse_operator(uccsdOperator)
# One could do uccsdOperator = jordan_wigner(uccsdOperator) before getting
#the sparse operator, but it's unnecessary. It's implicit that the 
#sparse matrix will be acting on the qubit space. The JW transform is performed
#by get_sparse_operator.

uccsdMatrix = scipy.sparse.linalg.expm(sparseOperator)

uccsdState = scipy.sparse.csc_matrix.dot(uccsdMatrix,hfState)

uccsdEnergy = exactStateEnergySparse(uccsdState,sparseHamiltonian)

### Number of excitation operators

This number sets the standard against which the size of the ansatze from Adapt-VQE should be compared.

In [12]:
operatorList = (list(uccsdOperator.get_operators()))

print("Number of excitation operators:",len(operatorList))

Number of excitation operators: 4


### Expectation estimation for the operator

In [13]:
# Test the expectation estimation for individual strings (measureString),
#for the Hartree Fock + UCCSD state preparation circuit with CCSD amplitudes.
# Applies UCCSD  circuit directly as a matrix (exponentiating the operator 
#obtained from OpenFermion as a sparse matrix).
# Optimized CCSD amplitudes are a very good starting point, because the CCSD 
#ansatz is the most important term in the UCCSD exponential series.

qubits = cirq.LineQubit.range(qubitNumber)
hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_electrons)]

statePreparationGates = cirq.Circuit(hfGates)
statePreparationGates.append(cirq.MatrixGate(uccsdMatrix.toarray()).on(*qubits))

repetitions = 1000
experimentalEnergyExpectation = 0

for string in formattedHamiltonian:
  
  expectationValue = measureString(string,repetitions,statePreparationGates,qubits)

  print("String:",string)
  
  print("Measured Expectation Value:",expectationValue)

  experimentalEnergyExpectation += expectationValue*formattedHamiltonian[string]
  
  bra = np.conj(uccsdState)
  ket = np.matmul(stringToMatrix(string),uccsdState)
  stringExpValue = np.real(np.dot(bra,ket))

  print("Calculated Expectation Value: {}\n".format(stringExpValue))

print("My measured energy:",experimentalEnergyExpectation)
  
print("My calculated energy:",uccsdEnergy)

String: IIII
Measured Expectation Value: 1.0
Calculated Expectation Value: 1.0

String: ZIII
Measured Expectation Value: -0.98
Calculated Expectation Value: -0.9744518905722033

String: IZII
Measured Expectation Value: -0.966
Calculated Expectation Value: -0.9744518905722033

String: IIZI
Measured Expectation Value: 0.98
Calculated Expectation Value: 0.9744518905722033

String: IIIZ
Measured Expectation Value: 0.976
Calculated Expectation Value: 0.9744518905722033

String: ZZII
Measured Expectation Value: 1.0
Calculated Expectation Value: 1.0

String: YXXY
Measured Expectation Value: -0.184
Calculated Expectation Value: -0.2245963333633451

String: YYXX
Measured Expectation Value: 0.132
Calculated Expectation Value: 0.2245963333633451

String: XXYY
Measured Expectation Value: 0.18
Calculated Expectation Value: 0.2245963333633451

String: XYYX
Measured Expectation Value: -0.23
Calculated Expectation Value: -0.2245963333633451

String: ZIZI
Measured Expectation Value: -1.0
Calculated Exp

## Trotterization

### *simulate_trotter*



#### Trotterizing the Hamiltonian

In [14]:
# Test trotterized circuit for time evolution

# Get a random initial state
random_seed = 8317
initialState = openfermion.haar_random_vector(
    2 ** qubitNumber, random_seed).astype(np.complex64)

# Initialize qubits
qubits = cirq.LineQubit.range(qubitNumber)

# Choose the evolution time
time = 10

# Get the exact evolution operator as a sparse matrix
evolutionMatrix = get_sparse_operator(-1j*qubitHamiltonian*time,qubitNumber)

# Calculate the exact evolved final state at the chosen time
exactFinalState = scipy.sparse.linalg.expm_multiply(evolutionMatrix,initialState)
                                                
print("Overlap of exact final state with...")

initialOverlap = np.around(calculateOverlap(initialState,exactFinalState),2)
print("...initial state:",initialOverlap)

# Simulate the circuit to obtain the final state for various numbers of trotter
#steps. More steps yield a final state closer to the exact state, as they
#should.
for steps in range(1,6):

  # Create trotterized time evolution circuit
  circuit = cirq.Circuit(simulate_trotter(qubits,hamiltonian,time,steps))

  # Simulate circuit on CIRQ 
  s = cirq.Simulator()
  results = s.simulate(circuit,initial_state = initialState)

  # Get the exact final wavefuntion
  finalState = results.final_state_vector

  overlap = np.around(calculateOverlap(exactFinalState,finalState),2)
  print("...final state from trotterization with {} steps: {}".format\
          (steps,overlap))

Overlap of exact final state with...
...initial state: 0.65
...final state from trotterization with 1 steps: 0.71
...final state from trotterization with 2 steps: 0.91
...final state from trotterization with 3 steps: 0.81
...final state from trotterization with 4 steps: 0.95
...final state from trotterization with 5 steps: 0.98


#### Trotterizing the UCCSD operator




In [15]:
# Trotterization attempts for the UCCSD operator
# Using CCSD amplitudes here. Since the point is testing the trotterization,
#it could be any. But CCSD should be a good first guess, as a starting point
#for the optimization after.

# 1: No trotterization (apply operator directly as a matrix on CIRQ for testing)
exactUccsdGates = cirq.MatrixGate(uccsdMatrix.toarray()).on(*qubits)

# 2: Use simulate_trotter function from OpenFermion, that directly gives a
#CIRQ operation tree.
# Does NOT work for the UCCSD operator. It's meant for Hamiltonians, so it 
#uses a low rank algorithm that assumes 8-fold symmetry.
# Could be adapted to the 4-fold mixed symmetry and anti-symmetry of UCCSD for 
#VQE, but not for operators in the Adapt-VQE pool.
# See arxiv:1808.02625.
# Like this, it only works for the single excitations. If one simulates the 
#circuit, |0011> is absent from the final wavefunction. That's because this
#computational basis state is a double excitation away from |1100>.
interactionOperator = get_interaction_operator(uccsdOperator,qubitNumber)
trotterUccsdGates = simulate_trotter(qubits,1j*interactionOperator,1)

# Choose operator to test
uccsdGates = trotterUccsdGates
uccsdCircuit = cirq.Circuit(uccsdGates)

In [16]:
qubits = cirq.LineQubit.range(qubitNumber)

# Append Hartree Fock state preparation circuit
hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_electrons)]
statePreparationGates = cirq.Circuit(hfGates)

# Append UCCSD circuit
statePreparationGates.append(uccsdCircuit)

# Simulate circuit (no sampling noise)
circuit = cirq.Circuit(statePreparationGates)
s = cirq.Simulator()
results = s.simulate(circuit)

# Get final state vector
finalState = results.final_state_vector

# Calculate final energy
finalEnergy = exactStateEnergySparse(results.final_state_vector,sparseHamiltonian)

# Calculate overlap.
# The Hartree Fock preparation circuit is always there, and UCCSD with CCSD
#amplitudes makes but minor adjustments to the energy.
# As such, it's important to take the overlap as compared to the overlap with 
#the Hartree Fock state.
# Overlap of the simulated state with the exact one should probably be bigger
#that with HF.

overlap1 = np.around(calculateOverlap(finalState,uccsdState),5)
print("\nOverlap of simulated state with exact state:",overlap1)

overlap2 = np.around(calculateOverlap(finalState,hfState),5)
print("Overlap of simulated state with HF state:",overlap2)

overlap3 = np.around(calculateOverlap(hfState,uccsdState),5)
print("Overlap of exact state with HF state:",overlap3)

# Print energies.
# Once again, it's important to compare with Hartree Fock energy, which is
#already close to the exact final energy. Simulated final energy should be 
#even closer.

print("\nHartree Fock Energy:",molecule.hf_energy)
print("Simulated Final Energy:",finalEnergy)
print("Exact Final Energy:",uccsdEnergy)


Overlap of simulated state with exact state: 0.99359
Overlap of simulated state with HF state: 1.0
Overlap of exact state with HF state: 0.99359

Hartree Fock Energy: -1.1167593073964255
Simulated Final Energy: -1.1167593073964257
Exact Final Energy: -1.137283458728186


In [17]:
# Alternative: test trotter ansatz given by simulate_trotter for UCCSD,
#starting from a random vector (rather than Hartree Fock).

interactionOperator = get_interaction_operator(uccsdOperator,qubitNumber)

trotterAnsatz = cirq.Circuit(simulate_trotter(qubits,interactionOperator,time = 1,n_steps = 1))

ansatzMatrix = get_sparse_operator(uccsdOperator,n_qubits = qubitNumber)
expMatrix = scipy.sparse.linalg.expm(ansatzMatrix)

random_seed = 683
initialState = openfermion.haar_random_vector(
    2 ** qubitNumber, random_seed).astype(np.complex64)

qubits = cirq.LineQubit.range(qubitNumber)
circuit = cirq.Circuit(trotterAnsatz)

# Alternative (for testing): apply exact exponentiation
#circuit = cirq.Circuit(cirq.MatrixGate(expMatrix.toarray()).on(*qubits))

s = cirq.Simulator()
results = s.simulate(circuit,initial_state = initialState)
finalTrotterEnergy = exactStateEnergySparse(results.final_state_vector,sparseHamiltonian)
#print(np.around(results.final_state_vector,decimals = 3))
#print(state)

initialEnergy = exactStateEnergySparse(initialState,sparseHamiltonian)

finalEnergy = energyFromExpOperator(uccsdOperator,initialState,sparseHamiltonian)

print("Initial Energy:",initialEnergy)
print("Theoretical final energy:",finalEnergy)
print("Simulated Final Energy:",finalTrotterEnergy)

Initial Energy: -0.21248124022928636
Theoretical final energy: -0.14824573681023198
Simulated Final Energy: -0.21248124022928636


### *trotterize_exp_qubop_to_qasm*



In [ ]:
def trotterizeQubitOperator(qubitOperator,qubits,time,steps): 
  
  qasmOps = openfermion.circuits.trotterize_exp_qubop_to_qasm\
    (qubitOperator, evolution_time = time,trotter_number = steps)

  qasmOps = list(qasmOps)

  gates = []

  for i in range(len(qasmOps)):

    qasmOp = qasmOps[i].split(" ")

    if qasmOp[0] == "Rz":
      # The OpenFermion function is off by a factor of two?
      # To yield correct results:
      #gate = cirq.rz(2*float(qasmOp[1])).on(qubits[int(qasmOp[2])])
      gate = cirq.rz(float(qasmOp[1])).on(qubits[int(qasmOp[2])])

    elif qasmOp[0] == "Rx":
      gate = cirq.rx(float(qasmOp[1])).on(qubits[int(qasmOp[2])])

    elif qasmOp[0] == "Ry":
      gate = cirq.ry(float(qasmOp[1])).on(qubits[int(qasmOp[2])])

    elif qasmOp[0] == "CNOT":
      gate = cirq.CX(qubits[int(qasmOp[1])],qubits[int(qasmOp[2])])

    elif qasmOp[0] == "H":
      gate = cirq.H(qubits[int(qasmOp[1])])
      
    elif qasmOp[0] == "C-Phase":
      gate = cirq.ops.CZPowGate(exponent = float(qasmOp[1])).on(qubits[int(qasmOp[2])],qubits[int(qasmOp[3])])

    gates.append(gate)

    gate = None

  return gates

#### Trotterizing the Hamiltonian

In [ ]:
# Test Hamiltonian time evolution by using the OpenFermion function that 
#outputs a circuit in QASM. This test is fine

# Choose a specific Slater Determinant as starting state (computational basis 
#ket, easy to prepare with CNOTs)
ket = [0,0,1,1]

# Convert the ket to a vector
initialState = fromKettoVector(ket)

# Choose a specific time at which to check the time evolution.
time = 1

# Get the exact evolution operator as a sparse matrix
evolutionMatrix = get_sparse_operator(-1j*qubitHamiltonian*time,qubitNumber)


# Calculate the exact evolved final state at the chosen time
exactFinalState = scipy.sparse.linalg.expm_multiply(evolutionMatrix,initialState)

print("Overlap of exact final state with...")

initialOverlap = np.around(calculateOverlap(initialState,exactFinalState),2)
print("...initial state:",initialOverlap)

# Simulate the circuit to obtain the final state for various numbers of trotter
#steps. More steps yield a final state closer to the exact state, as they
#should.
for steps in range(1,6):

  # Initialize qubits
  qubits = cirq.LineQubit.range(qubitNumber)

  # Create state preparation gates
  statePreparationGates = [cirq.X(qubits[i]) for i in range(qubitNumber) if ket[i]]

  # Get the trotter circuit as a list of CIRQ gates
  trotterGates = trotterizeQubitOperator(qubitHamiltonian,
                                         qubits,
                                         time,
                                         steps)

  # Prepare circuit (state preparation + trotter circuit)
  circuit = cirq.Circuit(statePreparationGates)
  circuit.append(trotterGates)

  # Simulate circuit
  s = cirq.Simulator()
  results = s.simulate(circuit)

  # Get the exact final vector
  finalState = results.final_state_vector

  # Calculate overlap with state from exact time evolution
  overlap = np.around(calculateOverlap(exactFinalState,finalState),2)
  print("...final state from trotterization with {} steps: {}".format\
          (steps,overlap))

#### Trotterizing the UCCSD operator

In [ ]:
# This one isn't fine. The factor of 2 in the rotation becomes relevant
#because the operator isn't a large sum of Pauli strings with small coefficients.

# Choose simple operator to test the function (anti-hermitian so that it is 
#exponentiated into a unitary operator)
operator = (FermionOperator(('3^ 0'))-FermionOperator(('0^ 3')))
qubitOperator = jordan_wigner(operator)

print("Fermion Operator:",operator)
print("Qubit Operator:",qubitOperator)

sparse = get_sparse_operator(operator)

expMatrix = scipy.sparse.linalg.expm(sparse)

state = scipy.sparse.csc_matrix.dot(expMatrix,hfState)

exactEnergy = exactStateEnergySparse(state,sparseHamiltonian)

qubits = cirq.LineQubit.range(qubitNumber)

hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_electrons)]

trotterGates = trotterizeQubitOperator(1j*qubitOperator,
                                       qubits,
                                       time=1,
                                       steps=1)

statePreparationGates = hfGates
statePreparationGates.append(trotterGates)

# For testing: calculate final state using the circuit matrix
#expMatrix = cirq.Circuit(trotterGates).unitary()
#state = scipy.dot(expMatrix,hfState)

# For testing: apply exact exponentiated operator on the circuit
#statePreparationGates.append(cirq.MatrixGate(expMatrix.toarray()).on(*qubits))

circuit = cirq.Circuit(statePreparationGates)
print(circuit)

s = cirq.Simulator()
results = s.simulate(circuit)
finalState = results.final_state_vector
finalEnergy = exactStateEnergySparse(finalState,sparseHamiltonian)

print("Final Simulated Energy:",finalEnergy)
print("Exact energy:",exactEnergy)

In [ ]:
# Test the function for the full UCCSD operator

qubits = cirq.LineQubit.range(qubitNumber)

hfGates = [cirq.X(qubits[i]) for i in range(electronNumber)]

qubitAnsatz = jordan_wigner(uccsdOperator)

trotterGates = trotterizeQubitOperator(1j*qubitAnsatz,
                                       qubits,
                                       time=1,
                                       steps=1)

circuit = cirq.Circuit(hfGates)
circuit.append(trotterGates)

# For testing: get unitary from trotter circuit and multiply it by the HF
#state to get the final state
#expMatrix = cirq.Circuit(trotterGates).unitary()
#state = scipy.dot(expMatrix,hfState)

# For testing: use exact exponentiated matrix instead of trotterized circuit
#statePreparationGates.append(cirq.MatrixGate(uccsdMatrix.toarray()).on(*qubits))

# Simulate circuit for exact results
s = cirq.Simulator()
results = s.simulate(circuit)

# Get the exact state vector at the end of the circuit
finalState = results.final_state_vector

# Calculate exact final energy
finalEnergy = exactStateEnergySparse(finalState,sparseHamiltonian)

overlap1 = np.around(calculateOverlap(finalState,uccsdState),5)
print("\nOverlap of simulated state with exact state:",overlap1)

overlap2 = np.around(calculateOverlap(finalState,hfState),5)
print("Overlap of simulated state with HF state:",overlap2)

overlap3 = np.around(calculateOverlap(hfState,uccsdState),5)
print("Overlap of exact state with HF state:",overlap3)

# Print energies.
# Once again, it's important to compare with Hartree Fock energy, which is
#already close to the exact final energy. Simulated final energy should be 
#even closer.

print("\nHartree Fock Energy:",molecule.hf_energy)
print("Simulated Final Energy:",finalEnergy)
print("Exact Final Energy:",uccsdEnergy)

### My trotterization function


In [ ]:
def trotterStep(operator,qubits,time): 
  '''
  Creates the circuit for applying e^(-j*operator*time), simulating the time
  evolution of a state under the Hamiltonian 'operator'.

  Arguments:
    operator (union[openfermion.QubitOperator, openfermion.FermionOperator,
      openfermion.InteractionOperator]): the operator to be simulated
    qubits ([cirq.LineQubit]): the qubits that the gates should be applied to
    time (float): the evolution time

  Returns:
    trotterGates (cirq.OP_TREE): the list of CIRQ gates that apply the 
      trotterized operator
  '''

  # If operator is an InteractionOperator, shape it into a FermionOperator
  if isinstance(operator,InteractionOperator):
    operator = get_fermion_operator(operator)

  # If operator is a FermionOperator, use the Jordan Wigner transformation
  #to map it into a QubitOperator
  if isinstance(operator,FermionOperator):
    operator = jordan_wigner(operator)
  
  # Get the number of qubits the operator acts on
  qubitNumber = count_qubits(operator)

  # Initialize list of gates
  trotterGates = []

  # Order the terms the same way as done by OpenFermion's 
  #trotter_operator_grouping function (sorted keys) for consistency.
  orderedTerms = sorted(list(operator.terms.keys()))

  # Add to trotterGates the gates necessary to simulate each Pauli string,
  #going through them by the defined order
  for pauliString in orderedTerms:

    # Get real part of the coefficient (the immaginary one can't be simulated,
    #as the exponent would be real and the operation would not be unitary).
    # Multiply by time to get the full multiplier of the Pauli string.
    coef = float(np.real(operator.terms[pauliString]))*time

    # Keep track of the qubit indices involved in this particular Pauli string.
    # It's necessary so as to know which are included in the sequence of CNOTs 
    #that compute the parity
    involvedQubits = []

    # Perform necessary basis rotations
    for pauli in pauliString:

      # Get the index of the qubit this Pauli operator acts on
      qubitIndex = pauli[0]
      involvedQubits.append(qubitIndex)

      # Get the Pauli operator identifier (X,Y or Z)
      pauliOp = pauli[1]

      if pauliOp == "X":
        # Rotate to X basis
        trotterGates.append(cirq.H(qubits[qubitIndex]))

      if pauliOp == "Y":
        # Rotate to Y Basis
        trotterGates.append(cirq.rx(np.pi/2).on(qubits[qubitIndex]))

    # Compute parity and store the result on the last involved qubit
    for i in range(len(involvedQubits)-1):

      control = involvedQubits[i]
      target = involvedQubits[i+1]

      trotterGates.append(cirq.CX(qubits[control],qubits[target]))
    
    # Apply e^(-i*Z*coef) = Rz(coef*2) to the last involved qubit
    lastQubit = max(involvedQubits)
    trotterGates.append(cirq.rz(coef*2).on(qubits[lastQubit]))

    # Uncompute parity
    for i in range(len(involvedQubits)-2,-1,-1):

      control = involvedQubits[i]
      target = involvedQubits[i+1]

      trotterGates.append(cirq.CX(qubits[control],qubits[target]))

    # Undo basis rotations
    for pauli in pauliString:

      # Get the index of the qubit this Pauli operator acts on
      qubitIndex = pauli[0]

      # Get the Pauli operator identifier (X,Y or Z)
      pauliOp = pauli[1]

      if pauliOp == "X":
        # Rotate to Z basis from X basis
        trotterGates.append(cirq.H(qubits[qubitIndex]))

      if pauliOp == "Y":
        # Rotate to Z basis from Y Basis
        trotterGates.append(cirq.rx(-np.pi/2).on(qubits[qubitIndex]))

  return trotterGates

def trotterizeOperator(operator,qubits,time,steps):
  '''
  Creates the circuit for applying e^(-j*operator*time), simulating the time
  evolution of a state under the Hamiltonian 'operator', with the given 
  number of steps. 
  Increasing the number of steps increases precision (unless the terms in the 
  operator commute, in which case steps = 1 is already exact).
  For the same precision, a greater time requires a greater step number
  (again, unless the terms commute)

  Arguments:
    operator (union[openfermion.QubitOperator, openfermion.FermionOperator,
      openfermion.InteractionOperator]): the operator to be simulated
    qubits ([cirq.LineQubit]): the qubits that the gates should be applied to
    time (float): the evolution time
    steps (int): the number of trotter steps to split the time evolution into

  Returns:
    trotterGates (cirq.OP_TREE): the list of CIRQ gates that apply the 
      trotterized operator
  '''
  
  trotterGates = []
  
  # Divide time into steps and apply the evolution operator the necessary 
  #number of times
  for step in range(1,steps+1):
    trotterGates += (trotterStep(operator,qubits,time/steps))
  
  return trotterGates

#### Testing

In [ ]:
# Test the function for the full UCCSD operator

qubits = cirq.LineQubit.range(qubitNumber)

hfGates = [cirq.X(qubits[i]) for i in range(electronNumber)]

trotterGates = trotterizeOperator(1j*uccsdOperator,
                                       qubits,
                                       time=1,
                                       steps=1)

circuit = cirq.Circuit(hfGates)
circuit.append(trotterGates)

# For testing: get unitary from trotter circuit and multiply it by the HF
#state to get the final state
#expMatrix = cirq.Circuit(trotterGates).unitary()
#state = scipy.dot(expMatrix,hfState)

# For testing: use exact exponentiated matrix instead of trotterized circuit
#statePreparationGates.append(cirq.MatrixGate(uccsdMatrix.toarray()).on(*qubits))

# Simulate circuit for exact results
s = cirq.Simulator()
results = s.simulate(circuit)

# Get the exact state vector at the end of the circuit
finalState = results.final_state_vector

# Calculate exact final energy
finalEnergy = exactStateEnergySparse(finalState,sparseHamiltonian)

overlap1 = np.around(calculateOverlap(finalState,uccsdState),5)
print("\nOverlap of simulated state with exact state:",overlap1)

overlap2 = np.around(calculateOverlap(finalState,hfState),5)
print("Overlap of simulated state with HF (starting) state:",overlap2)

# Print energies.
# Once again, it's important to compare with Hartree Fock energy, which is
#already close to the exact final energy. Simulated final energy should be 
#even closer.

print("\nHartree Fock Energy:",molecule.hf_energy)
print("Simulated Final Energy:",finalEnergy)
print("Exact Final Energy:",uccsdEnergy)

In [ ]:
# Check trotterized circuit for a simple operator
operator = QubitOperator('X0 Y1 Z2 Y3')
print("\n",cirq.Circuit(trotterizeOperator(operator,qubits,1,1)))

# Compare a larger trotterized circuit (the UCCSD ansatz circuit) with the 
#one obtained from OpenFermion's trotterize_exp_qubop_to_qasm, accounting for
#the factor of 2

operator = qubitAnsatz

qubits = cirq.LineQubit.range(qubitNumber)

unitary1 = cirq.Circuit(trotterizeOperator(operator,qubits,1,2)).unitary()
unitary2 = cirq.Circuit(trotterizeQubitOperator(2*operator,qubits,1,2)).unitary()

# Make sure that the circuit matrices are the same up to reasonable tolerance
tolerance = 10**(-10)

print("\nUp to a tolerance of",tolerance,end='')
if (unitary1 - unitary2 < tolerance).all():
  print (", circuit unitaries are equivalent.")
else:
  print(", circuit unitaries are not equivalent.")

In [ ]:
# Choose simple operator (anti-hermitian so that it is exponentiated into a 
#unitary operator)
operator = (FermionOperator(('3^ 0'))-FermionOperator(('0^ 3')))
qubitOperator = jordan_wigner(operator)

print("Fermion Operator:",operator)
print("Qubit Operator:",qubitOperator)

# Calculate exact result 
sparse = get_sparse_operator(operator)

expMatrix = scipy.sparse.linalg.expm(sparse)
state = scipy.sparse.csc_matrix.dot(expMatrix,hfState)

exactEnergy = exactStateEnergySparse(state,sparseHamiltonian)

qubits = cirq.LineQubit.range(qubitNumber)

hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_electrons)]

trotterGates = trotterizeOperator(1j*qubitOperator,
                                       qubits,
                                       time=1,
                                       steps=1)

statePreparationGates = hfGates
statePreparationGates.append(trotterGates)

# For testing: calculate the final state using the circuit matrix
#expMatrix = cirq.Circuit(statePreparationGates).unitary()
#state = scipy.dot(expMatrix,hfState)

# For testing: apply exact exponentiated operator
#statePreparationGates.append(cirq.MatrixGate(expMatrix.toarray()).on(*qubits))

circuit = cirq.Circuit(statePreparationGates)
print(circuit)

s = cirq.Simulator()
results = s.simulate(circuit)
finalState = results.final_state_vector
finalEnergy = exactStateEnergySparse(finalState,sparseHamiltonian)

print("Final Simulated Energy:",finalEnergy)
print("Exact energy:",exactEnergy)

## VQE

### UCCSD Energy Calculation Functions

#### Exact energy calculation

In [ ]:
def uccsdExactEnergy(packedAmplitudes,molecule):
    '''
    Calculates the exact energy of a state prepared by a given UCCSD circuit.

    Arguments:
      packedAmplitudes (list): the amplitudes that specify the parameters of the 
        UCCSD circuit
      molecule (openfermion.MolecularData): the molecule in consideration

    Returns:
      energy (float): the energy of the state prepared by this circuit

    '''

    hamiltonian = molecule.get_molecular_hamiltonian()
    qubitHamiltonian = jordan_wigner(hamiltonian)
    qubitHamiltonian.compress()

    electronNumber = molecule.n_electrons
    orbitalNumber = molecule.n_orbitals
    
    hfVector = jw_hartree_fock_state(electronNumber,orbitalNumber*2)

    qubitNumber = count_qubits(qubitHamiltonian)

    qubits = cirq.LineQubit.range(qubitNumber)

    uccsdOperator = openfermion.uccsd_singlet_generator(packedAmplitudes,
                                                        qubitNumber,
                                                        electronNumber)

    sparseOperator = openfermion.get_sparse_operator(uccsdOperator,qubitNumber)

    expMatrix = scipy.sparse.linalg.expm(sparseOperator)

    if sparseOperator.nnz:
      # The matrix has non null entries
      state = scipy.sparse.csc_matrix.dot(expMatrix,hfVector)

    else:
      state = hfState

    energy = exactStateEnergySparse(state,sparseHamiltonian)
    
    return energy

energy = uccsdExactEnergy(packedAmplitudes,molecule)
print("UCCSD starting energy (matrix algebra): {}".format(energy))
print("FCI (exact) energy: {}".format(molecule.fci_energy))

#### Trotterization (and simulation) energy

In [ ]:
def getUccsdEnergy(packedAmplitudes,molecule,shots,sample = True):
    '''
      Using the CIRQ simulator, samples the UCCSD state preparation circuit to 
    obtain an estimate for the energy.

    Arguments:
      packedAmplitudes (list): the amplitudes that specify the parameters of the 
        UCCSD circuit
      molecule (openfermion.MolecularData): the molecule in consideration
      shots (int): the number of circuit repetitions to be used
      sample (bool): if False, simulate the state vector and get the result
        free of sampling noise
    
    Returns:
      energy (float): an estimate for the energy of the state prepared by this 
        circuit

    '''

    hamiltonian = molecule.get_molecular_hamiltonian()
    qubitHamiltonian = jordan_wigner(hamiltonian)
    qubitHamiltonian.compress()
    formattedHamiltonian = convertHamiltonian(qubitHamiltonian)
    groupedHamiltonian = groupHamiltonian(formattedHamiltonian)

    qubitNumber = count_qubits(qubitHamiltonian)
    electronNumber = molecule.n_electrons

    uccsdOperator = openfermion.uccsd_singlet_generator(packedAmplitudes,
                                                        qubitNumber,
                                                        electronNumber)
    
    qubitOperator = jordan_wigner(uccsdOperator)

    qubits = cirq.LineQubit.range(qubitNumber)

    hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_electrons)]

    trotterGates = trotterizeOperator(1j*qubitOperator,
                                           qubits,
                                           time=1,
                                           steps=1)

    statePreparationGates = hfGates
    statePreparationGates.append(trotterGates)

    if sample:
      experimentalEnergy = 0

      # Obtain experimental expectation value for each necessary Pauli string by
      #calling the measureExpectation function, and perform the necessary weighed
      #sum to obtain the energy expectation value.
      for mainString in groupedHamiltonian:
          
          expectationValue = measureExpectation(mainString,\
                                              groupedHamiltonian[mainString],\
                                                  shots,statePreparationGates,qubits)
          experimentalEnergy+=expectationValue

      return experimentalEnergy

    else:

      circuit = cirq.Circuit(statePreparationGates)
      
      involvedQubits = list(circuit.all_qubits())

      # Make sure that the circuit acts on all qubits, so that the final state
      #vector has the correct dimension
      for qubit in qubits:
        if qubit not in involvedQubits:
          circuit.append(cirq.I(qubit))
      
      s = cirq.Simulator()
      results = s.simulate(circuit)
      finalState = results.final_state_vector

      calculatedEnergy = exactStateEnergySparse(finalState,sparseHamiltonian)
      return calculatedEnergy

shots = 1000
energy = getUccsdEnergy(packedAmplitudes,molecule,shots,sample = False)
print("UCCSD starting energy (trotter circuit): {}".format(energy))
print("FCI (exact) energy: {}".format(molecule.fci_energy))

### Optimization


#### Optimization function with Nelder Mead


In [ ]:
def vqe(packedAmplitudes,molecule,shots = 1000,optTolerance = (10**(-2),10**(-3)),\
        delta = 1.8,simulate = False,sample = False):
    '''
    Runs the VQE algorithm to find the ground state of a molecule, using
    UCCSD as the ansatz.

    Arguments:
      packedAmplitudes (list): the amplitudes that specify the starting 
        parameters of the UCCSD circuit
      molecule (openfermion.MolecularData): the molecule in consideration
      shots (int): the number of circuit repetitions to be used in the 
        expectation estimation
      optolerance (float,float): values of fatol and xatol that define the 
        accepted tolerance for convergence in the optimization
      simulate (bool): if False, the circuit will not be trotterized
      sample (bool): if False, the full state vector will be simulated and
        the result will be free of sampling noise
    '''
  
    global trackOptimization
    
    # Choose maximum number of function evaluations for the optimization
    # A lower number seems to work better when running the CIRQ simulator
    if(simulate):
        maxfev = 300
    else:
        maxfev = 1000
        
    fatol,xatol = optTolerance
    
    dim = len(packedAmplitudes)
    initialSimplex = np.array([packedAmplitudes,]*(dim+1))
    
    for i in range(dim):
        initialSimplex[i+1][i]+=delta
    
    # Select the options for the optimization
    options = {
        #"disp": True,
        "maxfev": maxfev, # Maximum function evaluations
        "fatol": fatol, # Acceptable absolute error in f for convergence
        "xatol": xatol, # Acceptable absolute error in xopt for convergence
        "adaptive": True,
        "initial_simplex": initialSimplex
        }
    
    # Choose whether to print the number of function evaluations as a 
    #percentage of the maximum allowed to inform on the stage of the 
    #optimization
    trackOptimization = False
    
    # Optimize the results from the CIRQ simulation
    if(simulate):
        optResults = scipy.optimize.minimize(getUccsdEnergy,
                                             packedAmplitudes,
                                             (molecule, shots,sample),
                                             method = 'Nelder-Mead',
                                             options = options)
                     
    # Optimize the results from analytical calculation
    else:
        optResults = scipy.optimize.minimize(uccsdExactEnergy,
                                             packedAmplitudes,
                                             (molecule),
                                             method = 'Nelder-Mead',
                                             options = options)
    trackOptimization = False
   
    return optResults

In [ ]:
findDelta = False

if findDelta:
  minError = 1

  for delta in np.linspace(0,2,21):

    print("Delta:",delta)

    vqeResult = vqe(randomPackedAmplitudes,molecule,simulate = False, delta=delta)
    vqeEnergy = vqeResult.fun
    error = vqeEnergy - molecule.fci_energy

    if error < minError:
      minError = error
      min = delta

    print("Error:",error)

  print("Value of delta that minimizes error:", min)

# Optimal: delta = 1.8

### Tests

#### Effect of the starting point

*Exact energy calculations, via matrix algebra, are used for all optimization starting points.*

*Two optimizers (Nelder-Mead / Cobyla) are tested.*

##### CCSD amplitudes

In [ ]:
initialEnergy = uccsdExactEnergy(packedAmplitudes,molecule)
print("Starting Energy:",initialEnergy) 

error = initialEnergy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))

print("\nNelder Mead:")
optResults = vqe(packedAmplitudes,molecule,simulate = False)

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

print("\nCobyla:")
optResults = scipy.optimize.minimize(uccsdExactEnergy,
                                     packedAmplitudes,
                                     (molecule),
                                     method = 'COBYLA')

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

Starting Energy: -2.860204605045257
Error: 5.175345476615689e-07
(in % of chemical accuracy: 0.032%)

Nelder Mead:
Error: 3.5105505613586274e-07
(in % of chemical accuracy: 0.022%)
Number of function calls: 52

Cobyla:
Error: 2.152154943146911e-08
(in % of chemical accuracy: 0.001%)
Number of function calls: 31


##### Random amplitudes

In [ ]:
randomPackedAmplitudes = [np.random.rand() for _ in range(len(packedAmplitudes))]

initialEnergy = uccsdExactEnergy(randomPackedAmplitudes,molecule)
print("Starting Energy:",initialEnergy) 
error = initialEnergy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))

print("\nNelder Mead:")
optResults = vqe(randomPackedAmplitudes,
                 molecule,
                 simulate = False)

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

print("\nCobyla:")
optResults = scipy.optimize.minimize(uccsdExactEnergy,
                                     randomPackedAmplitudes,
                                     (molecule),
                                     method = 'COBYLA',
                                     tol = 10**(-3),
                                     options={'rhobeg': 0.1})

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

Starting Energy: -0.941514756380886
Error: 1.9186903661989185
(in % of chemical accuracy: 120399.747%)

Nelder Mead:
Error: 2.134168397560643e-10
(in % of chemical accuracy: 0.000%)
Number of function calls: 189

Cobyla:
Error: 8.155600211168235e-06
(in % of chemical accuracy: 0.512%)
Number of function calls: 34


##### Null amplitudes

In [ ]:
nullPackedAmplitudes = np.zeros(len(packedAmplitudes))

initialEnergy = uccsdExactEnergy(nullPackedAmplitudes,molecule)

print("Starting Energy:",initialEnergy) 
error = initialEnergy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("\nNelder Mead:")
optResults = vqe(nullPackedAmplitudes,
                 molecule,
                 simulate = False,
                 sample = False)

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

print("\nCobyla:")

optResults = scipy.optimize.minimize(uccsdExactEnergy,
                                     nullPackedAmplitudes,
                                     (molecule),
                                     method = 'COBYLA',
                                     tol = 10**(-3),
                                     options={'rhobeg': 0.1})

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

Starting Energy: -2.8529210783245995
Error: 0.007284044255205124
(in % of chemical accuracy: 457.081%)

Nelder Mead:
Error: 4.5162121331543403e-07
(in % of chemical accuracy: 0.028%)
Number of function calls: 58

Cobyla:
Error: 4.747836130025718e-06
(in % of chemical accuracy: 0.298%)
Number of function calls: 22


#### Effect of Trotterization/Noise

*Nelder-Mead is used for all attempts here.*

*Always null starting amplitudes.*

##### Exact calculations

In [ ]:
initialEnergy = uccsdExactEnergy(nullPackedAmplitudes,molecule)
print("Starting Energy:",initialEnergy) 
error = initialEnergy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))

print("\nOptimization results (exact energy calculation):")
optResults = vqe(nullPackedAmplitudes,
                 molecule,
                 simulate = False)

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

Starting Energy: -2.8529210783245995
Error: 0.007284044255205124
(in % of chemical accuracy: 457.081%)

Optimization results (exact energy calculation):
Error: 4.5162121331543403e-07
(in % of chemical accuracy: 0.028%)
Number of function calls: 58


##### Trotterization

In [ ]:
initialEnergy = uccsdExactEnergy(nullPackedAmplitudes,molecule)
print("Starting Energy:",initialEnergy) 
error = initialEnergy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))

print("\nOptimization results (full simulation of the trotterized circuit):")
optResults = vqe(nullPackedAmplitudes,
                 molecule,
                 simulate = True,
                 sample = False)

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

Starting Energy: -2.8529210783245995
Error: 0.007284044255205124
(in % of chemical accuracy: 457.081%)

Optimization results (full simulation of the trotterized circuit):
Error: 3.3981030123086953e-07
(in % of chemical accuracy: 0.021%)
Number of function calls: 52


##### Trotterization and Noise

In [ ]:
initialEnergy = uccsdExactEnergy(nullPackedAmplitudes,molecule)
print("Starting Energy:",initialEnergy) 
error = initialEnergy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))

shots = 1000
print("\nOptimization results (sampling the trotterized circuit, {} shots):".format(shots))
optResults = vqe(nullPackedAmplitudes,
                 molecule,
                 shots = shots,
                 simulate = True,
                 sample = True)

functionCalls = optResults.nfev

energy = uccsdExactEnergy(optResults.x,molecule)
error = energy - molecule.fci_energy

print("Error:",error)
print("(in % of chemical accuracy: {:.3f}%)".format(error/chemicalAccuracy*100))
print("Number of function calls:",functionCalls)

Starting Energy: -2.8529210783245995
Error: 0.007284044255205124
(in % of chemical accuracy: 457.081%)

Optimization results (sampling the trotterized circuit, 1000 shots):
Error: 0.0005101992461704619
(in % of chemical accuracy: 32.016%)
Number of function calls: 67
